## Tensorflow training 1D potential example


Since this is a bit more advanced than the sklearn we recommend you start the other one first

In [7]:
"""First we import our dataset examples"""
import MLTSA
potentials = MLTSA.datasets.one_d_pot
dataset = MLTSA.datasets.one_d_dataset
#import MLTSA.datasets.one_d_pot as potentials
#import MLTSA.datasets.one_d_dataset as dataset
import matplotlib.pyplot as plt
import numpy as np

#This sets the potentials, don't re-run
total_n_pots = 25
n_DW = 5
relevant_DW_n = 2
#After defining the desired parameters we define the potentials accordingly
pots = potentials(total_n_pots, n_DW, relevant_DW_n)
# This creates the first dataset of data.
# It creates the mixing coefficients don't re-run
n_features = 180
degree_of_mixing = 2
#We specified the number of features wanted and how much they will mix
oneD_dataset = dataset(pots, n_features, degree_of_mixing)


Defining Potentials: 100%|##########| 25/25 [00:00<00:00, 2005.00it/s]


This has set up our dataset for further use, since TensorFlow is more scalable and compatible with GPU calculations, we will do a more extensive search on this example. 

Let's generate the actual linear mixed data we will use for training.   


In [8]:
#Generate the trajectories
n_simulations = 100
n_steps = 500
data, ans = oneD_dataset.generate_linear(n_simulations, n_steps)
data_val, ans_val = oneD_dataset.generate_linear(int(n_simulations/2), n_steps)

#Prepare it for training
time_frame = [30, 60] #Same time frame as the sklearn one
X, Y = oneD_dataset.PrepareData(data, ans, time_frame, mode="Normal")
X_val, Y_val = oneD_dataset.PrepareData(data_val, ans_val, time_frame, mode="Normal")

Generating dataset


Running Simulations: 100%|##########| 100/100 [00:20<00:00,  4.96it/s]


Getting simulation labels for the generated data


Classifying Simulation Outcomes: 100%|##########| 100/100 [00:00<00:00, 490561.87it/s]


Generating dataset


Running Simulations: 100%|##########| 50/50 [00:09<00:00,  5.02it/s]


Getting simulation labels for the generated data


Classifying Simulation Outcomes: 100%|##########| 50/50 [00:00<00:00, 349525.33it/s]



Now that we have got the X and Y ready to fit to, we will train the model, in this example we want to train a Multi-Layer Perceptron just like the one in Sklearn, but we will use TensorFlow instead.


In [9]:
#Luckily we have an MLP set up to use
import  MLTSA_tensorflow.TF_old_MLP as MLP_tf
from sklearn.model_selection import train_test_split

#Let's do the splits to validate our training
# data = [x_train, x_test, y_train, y_test]
epoch_data = train_test_split(X, Y, test_size=0.4, stratify=Y) # We'll train on 40% of the data

acc_train, acc_test, acc_val, loss = MLP_tf.tf_train(batch_size=[time_frame[1]-time_frame[0]], data=epoch_data, validation=[X_val, Y_val], mode="notebook")

SyntaxError: invalid syntax (<ipython-input-9-6387412fc4d8>, line 2)

In [ ]:
epoch_data



The previous training has generated a record of the training, test and validation accuracy as well as the evolution of the loss throughout the epochs. 

These can be plotted to see how the model is learning the outcome. Additionally, a checkpoint of our trained model has been created and saved for us at "model_best_accuracy.ckpt"


In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].plot(acc_train*100, color="r", label="Train")
ax[0].plot(acc_test*100, color="b", label="Test")
ax[0].plot(acc_val*100, color="g", label="Validation")
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(loss, label="training loss")
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("loss")


Now we can move onto trying the MLTSA for this ML model. For that we call the MLTSA() method within the tensorflow package.
